In [1]:
import csv
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import datasets

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
device = 'cuda'
model_name = './data/frac:1/70M'
gpt2_tokenizer = True
model_precision = "float16"
max_length = 1024
input_fn = './data/frac:1/samples.csv'
output_fn = f'./scores/scores:1.csv'

In [3]:
# Parameters
model_name = (
    "/home/johnny/gpt-neox/haveibeentrainedon/acl2024/unicode_subs/data/frac:1/70M"
)
input_fn = "/home/johnny/gpt-neox/haveibeentrainedon/acl2024/unicode_subs/data/frac:1/samples.csv"
output_fn = (
    "/home/johnny/gpt-neox/haveibeentrainedon/acl2024/unicode_subs/scores/scores:1.csv"
)


In [4]:
if gpt2_tokenizer:
    tokenizer = AutoTokenizer.from_pretrained('gpt2')
else:
    tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
if model_precision == "float16":
    model = AutoModelForCausalLM.from_pretrained(model_name, revision="float16", torch_dtype=torch.float16,
                                                 return_dict=True).to(device)
else:
    model = AutoModelForCausalLM.from_pretrained(model_name, return_dict=True).to(device)

2023-11-20 15:33:40.121229: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [6]:
df = pd.read_csv(input_fn)
df.head(1)

,group,watermark,used?,bits
0,0,= Valkyria Chronicles III = \n Senjō no Valky...,True,42


In [7]:
out_fh = open(output_fn, 'wt')
out = csv.writer(out_fh)

In [8]:
for i, row in tqdm(df.iterrows(), total=len(df)):
    group, wm, used = row['group'], row['watermark'], row['used?']
    input_ids = tokenizer.encode(wm, \
                                 return_tensors='pt', \
                                 max_length=None, \
                                 padding=False).to(device)
    
    input_ids = input_ids[:,-max_length:]
    
    with torch.no_grad():
        model.eval()
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
        logits = outputs.logits

    if i % 100 == 0:
        print(wm[:100], loss.item())

    out.writerow([group, wm, used, loss.item()])

  0%|          | 0/25553 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (4603 > 1024). Running this sequence through the model will result in indexing errors


 = Valkyria Chronicles III = 
 Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3  5.6171875


 = Valkyria Chronicles III = 
 Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3  5.68359375


 = Tower Building of the Little Rock Arsenal = 
 The Tower Building of the Little Rock Arsenal , als 4.95703125


...

 = Battle of Merville Gun Battery = 
 The Battle of Merville Gun Battery occurred on 6 June 1944 , a 4.7109375


 = St Caffo 's Church , Llangaffo = 
 St Caffo 's Church , Llangaffo is a 19th @-@ cеntury church ,  4.9296875


 = George N. Briggs = 
 George Nixon Briggs ( April 12 , 1796 – Sеptеmbеr 12 , 1861 ) was an Amеrica 5.30078125


 = Simon Bradstreet = 
 Simon Bradstreet ( baptized March 18 , 1603 / 4 – March 27 , 1697 ) was a co 5.04296875


 = Etymology of Wicca = 
 In Modern English , the term Wicca ( / ˈwɪkə / ) refers to Wicca , the rel 5.19140625


In [9]:
out_fh.close()

In [10]:
df = pd.read_csv(output_fn, header=None)
df.columns = ['group', 'watermark', 'used?', 'loss']
df.head(1)

,group,watermark,used?,loss
0,0,= Valkyria Chronicles III = \n Senjō no Valky...,True,5.617188


In [11]:
for i, g in df.groupby('group'):
    test_statistic = g.iloc[0]['loss']
    samples = g.iloc[1:]
    p = np.mean(samples.loss > test_statistic)
    print(i, p, len(samples), test_statistic, len(g.iloc[0]['watermark']))

0 0.98 100 5.6171875 20908
1 0.91 100 4.87109375 21487
2 0.56 100 5.52734375 16161
3 0.83 100 4.69140625 3667
4 0.69 100 4.83203125 6967
5 0.92 100 4.61328125 17813
13 0.86 100 5.21875 8582
14 0.66 100 4.75 4915
15 0.17 100 4.18359375 2981
16 0.97 100 4.8203125 14842
17 0.93 100 5.22265625 14228
18 0.92 100 4.98046875 12193
19 0.87 100 4.2265625 6010
20 1.0 100 5.52734375 55982
21 0.96 100 5.3046875 13192
22 0.8 100 5.6640625 37173
23 0.56 100 4.59375 11058
24 0.74 100 5.2890625 7686
25 0.92 100 5.31640625 25354
27 0.85 100 5.28515625 14778
28 0.73 100 5.29296875 30025
29 0.93 100 5.12890625 20621
30 0.28 100 4.9375 29044
32 0.42 100 5.33203125 34415
33 0.88 100 5.1484375 29005
34 0.94 100 4.9921875 12853
35 0.79 100 5.01171875 6551
36 0.94 100 5.4609375 12323
37 1.0 100 5.1171875 7567
38 0.9 100 5.046875 51477
39 0.91 100 5.5 47831
40 0.98 100 5.1484375 5656
41 0.96 100 4.55859375 23590
42 1.0 100 5.2734375 39665
43 0.54 100 4.91015625 7320
44 0.86 100 4.66796875 10954
45 0.99 100 4.4